In [ ]:
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from nltk.tokenize import WordPunctTokenizer

# Data base import

In [ ]:
from utils import sample_data

# -- Get the data -- #
NB_SAMPLES = 10
PATH_SAMPLE = Path("../data/samples/sample_{}_train.csv".format(NB_SAMPLES))
DATA_SAMPLE = pd.read_csv(PATH_SAMPLE)

In [ ]:
print(DATA_SAMPLE)

# Pre-processing

In [ ]:
from descriptors.glove_descriptor.glove_descriptor import descriptor

# -- Parameters -- #
WORD_SIZE = 50  # 50 or 100 or 200 or 300
SENTENCE_SIZE = 50  # What ever
FILL_WITH = 0  # Should be an integer
SPLIT_PUNCTUATION = False # to tell wheter the puncutation "!?.;,/" etc are keeped sticked to a word or not


# -- Get the descriptor -- #
PATH_DESCRIPTOR = Path("../data/glove_descriptor/glove.6B.{}d.txt".format(WORD_SIZE))
DESCRIPTIONS = pd.read_csv(PATH_DESCRIPTOR, sep=" ", header=None)


# -- Describe every tweet -- #
(X_STRING, X_SCALAR) = descriptor(DATA_SAMPLE["text"], DESCRIPTIONS, SENTENCE_SIZE, WORD_SIZE, FILL_WITH)

# -- Get the labels -- #
IMPORTANT_WORDS = np.random.randint(2, size=(NB_SAMPLES, SENTENCE_SIZE))


print("\nOriginal data :")
print(X_STRING[0], "\nlen(X_STRING)", len(X_STRING))
print("\nDescriptor :")
print(X_SCALAR[0], "\nX_SCALAR.shape", X_SCALAR.shape)
print("\nLabel :")
print(IMPORTANT_WORDS[0], "\nIMPORTANT_WORDS.shape", IMPORTANT_WORDS.shape)

# Classification

In [ ]:
from sklearn.neighbors import KNeighborsRegressor

In [ ]:
nb_neighbors = 2

knn = KNeighborsRegressor(nb_neighbors, weights="distance")
knn.fit(X_SCALAR, IMPORTANT_WORDS)

In [ ]:
from utils.post_processing import pred_to_sentence, sentence_to_string, filter_character

pred = knn.predict(X_SCALAR)
meaning_sentences = pred_to_sentence(X_STRING, pred)

results = []
for sentence in meaning_sentences:
    result = ""
    for word in sentence:
        filtered_word = filter_character(word, "$")
        if len(filtered_word) != 0:
            result += filtered_word + " "
    results.append(result)
results=np.array(results, dtype=object)

In [ ]:
print(results)

In [ ]:
print(DATA[:, 2])

In [ ]:
from utils.loss import jaccard

avg = 0
for i in range(len(results)):
    print(avg)
    avg += jaccard(results[i], SMALL_DATA.sample_data["selected_text"].to_numpy()[i])
avg /= len(results)

print(avg)